## 소개


이 샘플은 환자가 심장병에 걸렸는지를 알아내는 데 사용되는 이진 분류(binary classification) 알고리즘 입니다. 이 예에서는 UCI 저장소에서 가져온 Cleveland Heart Disease 데이터 세트에서 샘플 데이터를 업로드합니다. 데이터 세트는 303개의 개별 데이터로 구성됩니다. 컬럼 설명 및 샘플 데이터는 데이터 저장소를 참조하십시오:
https://archive.ics.uci.edu/ml/datasets/heart+Disease 샘플 데이터를 다운로드하고 데이터를 S3 버킷에 저장한 다음 이 노트북의 셀을 실행하여 자체 모델을 구축하고 배포할 수 있습니다.

이 튜토리얼의 나머지 부분에서는 이진 분류(binary classification) 알고리즘을 사용하여 심장병을 예측하는 방법에 대해 설명합니다



필요한 Python 라이브러리를 import 합니다. 

In [1]:
import os                                         # For manipulating filepath names  
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import io
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.

import sagemaker                                  # Amazon SageMaker's Python SDK provides many helper functions
from sagemaker import get_execution_role          # Define IAM role
import sagemaker.amazon.common as smac
import boto3

## 설정

다음을 설정하는 것으로 시작 해보겠습니다.

* SageMaker Role은 데이터에 대한 학습 및 호스팅 액세스 권한을 부여하는 데 사용됩니다. <br />
  아래 코드는 SageMaker 노트북 인스턴스에서 사용하는 것과 동일한 역할을 사용합니다. 그렇지 않은 경우 SageMakerFullAccess 정책이 연결된 역할의 전체 ARN을 지정합니다.
* 모델 객체를 훈련하고 저장하는 데 사용하려는 S3 버킷 지정

In [2]:
#버킷명 입력
#bucket = '{ENTER_BUCKET_NAME}' # 20220422-sagemaker-workshop-jbuh-kwpark
bucket = '20220422-sagemaker-workshop-jbuh-kwpark' # 20220422-sagemaker-workshop-jbuh-kwpark
prefix = 'sagemaker/heart-disease'

#데이터 파일명 입력 (ex. heart.csv)
data_key = 'heart.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
 
# IAM role 설정
import re
role = get_execution_role()

### 데이터 수집

 데이터를 수집하기 전에 UCI 저장소의 데이터 세트(예: heart_data.csv)가 S3 버킷에 업로드되었는지 확인하십시오. 기본적으로 SageMaker 역할은 'sageMaker*'로 시작하는 버킷에 액세스할 수 있습니다. 아래 코드는 지정된 S3 버킷에서 데이터를 읽고 샘플을 출력합니다.


In [3]:
# S3로부터 데이터 읽기
heart_data = pd.read_csv(data_location)

#샘플 데이터 출력
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


### 데이터 변환

알고리즘에는 특정 input 및 output 요구사항이 있기 때문에 데이터 세트를 변환하는 것도 데이터 사이언티스트가 학습을 시작하기 전에 거쳐야 하는 프로세스의 일부입니다. Amazon SageMaker의 Linear Learner 알고리즘은 recordIO-wrapped protobuf 포맷을 지원합니다.

아래 코드는 다음을 수행합니다:

데이터를 가져와서 Numpy array로 변환합니다. SageMaker Linear Learner 알고리즘이 기대하는 float32 유형이어야 합니다.

Linear Learner 알고리즘에는 observations를 나타내는 행과 feature의 dimensions 을 나타내는 열이 있는 데이터 매트릭스가 필요합니다. 또한 데이터  요소와 일치하는 라벨이 들어 있는 추가 열이 필요합니다.

input의 경우 모델에 라벨이 지정된 예제(x, y)를 제공합니다. x는 고차원 벡터이고 y는 숫자 라벨입니다. 이진 분류 문제의 경우 라벨은 0 또는 1이어야 합니다.

The Linear Learner 알고리즘은 특성 행렬(features matrix) 과 라벨 벡터(labels vector)가 필요합니다.

라벨 열은 'target' 열입니다. 이 경우 마지막 열을 예측하여 사용자에게 심장병이 있는지 확인하려고 합니다. 값이 1이면 심장병이 있음을 나타내고 0이면 그렇지 않습니다.

In [4]:
vectors = np.array(heart_data).astype('float32')

#target 열 - 값은 0 이나 1 이어야 함
labels = vectors[:,13]
print ("label data is")
print (labels)


#target 열을 제거. 이 피쳐는 훈련 데이터의 일부로 사용
training_data = vectors[:, :13]
print ("Training data is")
print (training_data)



label data is
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Training data is
[[63.  1.  3. ...  0.  0.  1.]
 [37.  1.  2. .

S3에 학습 데이터를 업로드해 보겠습니다.

In [5]:
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, training_data, labels)
buf.seek(0)

key = 'recordio-pb-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))




uploaded training data location: s3://20220422-sagemaker-workshop-jbuh-kwpark/sagemaker/heart-disease/train/recordio-pb-data


## Artifact 학습
데이터가 학습되면 다음 위치에 업로드됩니다.

In [6]:
output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))

training artifacts will be uploaded to: s3://20220422-sagemaker-workshop-jbuh-kwpark/sagemaker/heart-disease/output


## Linear 모델 학습

데이터를 사전 처리하고 올바른 학습 포맷으로 사용할 수 있게 되면 다음 단계는 데이터를 사용하여 실제로 모델을 훈련하는 것입니다. 이 데이터는 상대적으로 작기 때문에 Linear Learner 학습 알고리즘의 성능을 과시하기 위한 것이 아닙니다.

Amazon SageMaker Python SDK를 사용하여 학습을 시작하고 완료될 때까지 상태를 모니터링합니다. 이 예에서는 약 7분에서 11분 정도 걸립니다. 데이터 세트가 작지만, 하드웨어 프로비저닝 및 알고리즘 컨테이너 로드에 약간의 시간이 소요됩니다.

우리는 이진 분류를 수행하고(환자가 심장 질환을 가지고 있는지 여부), 지정된 컴퓨팅(예: c5.xlarge)에서 모델을 훈련하고, 훈련 세트의 features 또는 dimiensions을 세분화합니다.

### SageMaker 의 linear-learner 를 훈련하고 호스팅하기 위해 사용되는 컨테이너 이미지 지정

SageMaker 의 ECR 컨테이너를 통해 해당되는 built-in 알고리즘을 사용할 수 있습니다.

In [7]:
#from sagemaker.amazon.amazon_estimator import get_image_uri
#import sagemaker
from sagemaker import image_uris

container = image_uris.retrieve(framework="linear-learner", 
                                region=boto3.Session().region_name)

다음으로 파라미터를 지정하여 esitmator를 생성합니다.

- built-in 알고리즘 컨테이너 사용
- 사용할 IAM 역할(role)
- 학습용 인스턴스 타입과 수량 ('local_cpu'를 사용해 해당 노트북의 인스턴스 내에서 학습을 진행할 수도 있습니다.)
- 출력 데이터를 위한 S3위치
- 알고리즘 하이퍼파라미터

이제 다음 파라미터를 이용하여 .fit() 명령을 실행합니다.

- 학습용(train)/검증용(validation) 데이터가 있는 S3 위치

본 예제는 학습과 검증 데이터셋을 모두 사용하므로 두 채널을 모두 지정합니다. Trainin job을 수행하기 위해서 학습용 서버가 생성되는데에 5분정도 소요됩니다.

데이터 학습을 수행할 시에 발생하는 과금은, EC2 인스턴스의 생성 시간이 제외된 데이터를 학습하는 시간만 요금이 부과됩니다. Log의 마지막 부분에 표시되는 Training seconds와 Billable seconds를 참고하셔서 과금이 발생한 시간을 참고하실 수 있습니다.

- Training seconds: Training job을 실행한 실제 컴퓨팅 학습 시간
- Billable seconds: Spot 할인이 적용된 후 청구되는 시간

In [8]:
sess = sagemaker.Session()
linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       instance_count=1, 
                                       instance_type='ml.c5.xlarge',
                                       output_path=output_location,
                                       sagemaker_session=sess)
linear.set_hyperparameters(feature_dim=13,
                           predictor_type='binary_classifier',
                           mini_batch_size=100)

linear.fit({'train': s3_train_data})

2022-04-21 18:40:31 Starting - Starting the training job...
2022-04-21 18:40:54 Starting - Preparing the instances for trainingProfilerReport-1650566431: InProgress
......
2022-04-21 18:42:02 Downloading - Downloading input data
2022-04-21 18:42:02 Training - Downloading the training image.....Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/21/2022 18:42:42 INFO 140680991950656 integration.py:636] worker started
[04/21/2022 18:42:42 INFO 140680991950656] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimiz

## 모델 호스팅

이제 모델을 학습시켰으므로 Amazon SageMaker의 실시간 호스팅 엔드포인트에 모델을 배포할 수 있습니다. 이렇게 하면 모델에서 동적으로 예측(또는 추론)할 수 있습니다. Endpoint 생성은 5-10분 정도 소요됩니다.

- 호스팅에 사용할 EC2 인스턴스 유형
- 인스턴스의 초기 갯수
- 호스팅 모델명

In [9]:
%%time

endpoint_name='heart-disease-'+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
heartdisease_predictor = linear.deploy(initial_instance_count=1,
                           instance_type='ml.c5.xlarge',
                           endpoint_name=endpoint_name)

----!CPU times: user 79.2 ms, sys: 8.66 ms, total: 87.9 ms
Wall time: 2min 1s


## 모델 검증(예측)

마지막으로 모델을 사용할 수 있는지 검증할 수 있습니다. HTTP POST 요청을 엔드포인트로 전달하여 예측을 가져올 수 있습니다. 이 작업을 더 쉽게 하기 위해 Amazon SageMaker Python SDK를 다시 사용하고 알고리즘에 맞는 요청을 직렬화(serialize)하고 응답을 역직렬화(deserialize)하는 방법을 지정합니다.

In [10]:
#from sagemaker.predictor import csv_serializer, json_deserializer
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

heartdisease_predictor.serializer = CSVSerializer()
heartdisease_predictor.deserializer = JSONDeserializer()

In [11]:
print('Endpoint name: {}'.format(heartdisease_predictor.endpoint_name))

Endpoint name: heart-disease-2022-04-21-18-43-13


다음 샘플 데이터를 테스트용으로 전달해 보겠습니다. 파일의 단일 레코드입니다.

In [12]:
vectors[5][0:13]

array([ 57. ,   1. ,   0. , 140. , 192. ,   0. ,   1. , 148. ,   0. ,
         0.4,   1. ,   0. ,   1. ], dtype=float32)

In [13]:
result = heartdisease_predictor.predict(vectors[5][0:13])
print(result)

{'predictions': [{'score': 0.8270459771156311, 'predicted_label': 1}]}


## Endpoint 삭제

In [14]:
import sagemaker

heartdisease_predictor.delete_endpoint()